# OBSPY

In [ ]:
client = Client("IRIS")
inventory = client.get_stations(
    minlongitude=6.0,
    maxlongitude=19.0,
    minlatitude=35.0,
    maxlatitude=47.0,    
    channel="BHZ",
    level="response"
)

In [ ]:
from obspy import UTCDateTime
def extract_traces(inventory, t0, duration_seconds, sampling_rate):
    t1 = t0 + duration_seconds
    traces = []
    for net in inventory:
        for sta in net:

            # --- controllo periodo operativo (robusto) ---
            if sta.start_date and sta.start_date > t0:
                continue
            if sta.end_date and sta.end_date < t1:
                continue

            try:
                # wildcard su location
                st = client.get_waveforms(
                    net.code,
                    sta.code,
                    "*",
                    "BHZ",
                    t0,
                    t1,
                    attach_response=True
                )

                # se arrivano più location, scegli una (es. "00" se c'è)
                if len(set(tr.stats.location for tr in st)) > 1:
                    if "00" in [tr.stats.location for tr in st]:
                        st = st.select(location="00")
                    else:
                        st = st[:1]  # prendine una e basta

                #print(f"SUCCESS {net.code}.{sta.code}.{st[0].stats.location}.BHZ")


            except Exception as e:
                continue

            trp = st[0].copy()

            trp.detrend("linear")
            trp.taper(max_percentage=0.05, type="hann")  # 5% cosine taper at both ends
            
            #trp.detrend("demean")
            #trp.taper(0.02)
#
            #trp.remove_response(
            #    inventory=inventory,
            #    output="VEL",
            #    pre_filt=(0.2, 0.5, 15, 20),
            #)

            #trp.filter("bandpass", freqmin=0.5, freqmax=15.0, corners=4, zerophase=True)
            f_nyq = trp.stats.sampling_rate / 2
            freqmax = min(10, 0.9 * f_nyq)  # 90% of Nyquist
            trp.filter("bandpass", freqmin=2.0, freqmax=freqmax, corners=4, zerophase=True)
            trp.resample(sampling_rate)


            x = trp.data.astype(np.float32)
            traces.append(x)
    return traces

In [ ]:
duration = 60*10
sampling_rate = 50.0  # Hz
t0 = UTCDateTime("2016-08-24T01:35:00")
t1 = t0 + duration
traces = extract_traces(inventory, t0, duration, sampling_rate)

In [ ]:
def two_timeseries_lagged_infos(x1, x2, window_size, lag):
    """
    x1, x2: 1D numpy arrays of the same length
    window_size: size of the moving window
    lag: lag between the two time series (in samples)
    """
    n = len(x1)
    assert len(x2) == n, "Time series must have the same length"
    assert abs(lag) < n, "Lag must be less than the length of the time series"

    infos = []
    for start in range(0, n - window_size + 1):
        end = start + window_size
        segment1 = x1[start:end]
        segment2 = x2[start + lag:end + lag]
        if(len(segment2) != window_size):
            continue


        infos.append(np.cov(segment1, segment2))
    return infos

In [ ]:
window_size = int(10.0 * sampling_rate)
lag = int(3.0 * sampling_rate)
results= []
variance_of_mutuals = []
mutuals = []
for info in two_timeseries_lagged_infos(traces[0], traces[3], window_size, lag):
    variance_of_mutuals.append(info[0,1]**2 / (info[0,0]*info[1,1]))
    mutuals.append(-0.5*np.log(1-variance_of_mutuals[-1]))
    vals = np.linalg.eigvalsh(info)
    results.append(vals)
results = np.array(results)
variance_of_mutuals = np.array(variance_of_mutuals)
mutuals = np.array(mutuals)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(12, 4))

axs[0].plot(variance_of_mutuals/mutuals)
axs[1].plot(traces[0], color='red', alpha=0.5)
axs[1].plot(traces[1], color='blue', alpha=0.5)
plt.show()